In [1]:
levels = 5

lulcmap = 'cosc2021'

db = 'gthruth_valeflores_v3'

harmtbl = {
    'TABLE'     : '/mnt/d/autocls_e4/harmonizacao_classes_cosc21.dbf',
    'MAPID'     : 'coscid',
    'MAPNAME'   : 'coscname',
    'CLASSID'   : 'idclcplus',
    'CLASSNAME' : 'clcplus'
}

out_tbl = '/mnt/d/autocls_e4/ground_truth_exp4.xlsx'

In [2]:
from glass.sql.q import q_to_obj
from glass.rd import tbl_to_obj
from glass.wt import obj_to_tbl

In [3]:
hdf = tbl_to_obj(harmtbl['TABLE'])

PyTables is not installed. No support for HDF output.


In [4]:
hdf['refname'] = 'CASE WHEN code = \'' + hdf[harmtbl['MAPID']] + '\' THEN \'' + hdf[harmtbl['CLASSNAME']] \
    + '\''

hdf['refid'] = 'CASE WHEN code = \'' + hdf[harmtbl['MAPID']] + '\' THEN \'' + hdf[harmtbl['CLASSID']].astype(str) \
    + '\''

In [5]:
refid   = hdf['refid'].str.cat(sep=" ELSE ") + " END " * 15 + 'AS refid'
refname = hdf['refname'].str.cat(sep=" ELSE ") + " END " * 15 + 'AS refname'

In [6]:
hdf

,clcplus,idclcplus,cosc,coscname,coscid,refname,refid
0,Sealed Surface,1,100,Artificializado,100,CASE WHEN code = '100' THEN 'Sealed Surface',CASE WHEN code = '100' THEN '1'
1,Agriculture,2,211,Culturas anuais de outono/inverno,211,CASE WHEN code = '211' THEN 'Agriculture',CASE WHEN code = '211' THEN '2'
2,Agriculture,2,212,Culturas anuais de primavera/verão,212,CASE WHEN code = '212' THEN 'Agriculture',CASE WHEN code = '212' THEN '2'
3,Agriculture,2,213,Outras áreas agrícolas,213,CASE WHEN code = '213' THEN 'Agriculture',CASE WHEN code = '213' THEN '2'
4,Tree cover,3,311,Sobreiro e Azinheira,311,CASE WHEN code = '311' THEN 'Tree cover',CASE WHEN code = '311' THEN '3'
5,Tree cover,3,312,Eucalipto,312,CASE WHEN code = '312' THEN 'Tree cover',CASE WHEN code = '312' THEN '3'
6,Tree cover,3,313,Outras folhosas,313,CASE WHEN code = '313' THEN 'Tree cover',CASE WHEN code = '313' THEN '3'
7,Tree cover,3,321,Pinheiro bravo,321,CASE WHEN code = '321' THEN 'Tree cover',CASE WHEN code = '321' THEN '3'
8,Tree cover,3,322,Pinheiro manso,322,CASE WHEN code = '322' THEN 'Tree cover',CASE WHEN code = '322' THEN '3'
9,Tree cover,3,323,Outras resinosas,323,CASE WHEN code = '323' THEN 'Tree cover',CASE WHEN code = '323' THEN '3'


In [7]:
lquery = []

for i in range(1, levels):

    if i == 1:
        q = (
            f"SELECT cid AS root{str(i+1)}, rootid "
            "FROM gthruth_fishnet AS tbl "
            f"WHERE tbl.level = {str(i+1)}"
        )

    else:
        q = (
            f"SELECT tbl.cid AS root{str(i+1)}, jtb.rootid, "
            "ST_Area(tbl.geom) AS geom "
            "FROM gthruth_fishnet AS tbl "
            f"INNER JOIN ({lquery[i-2]}) AS jtb "
            f"ON tbl.rootid = jtb.root{str(i)} "
            f"WHERE tbl.level = {str(i+1)}"
        )

    lquery.append(q)

flq = []

cls_view = (
    "SELECT * "
    "FROM pxclasses AS pxcls "
    "INNER JOIN ("
        "SELECT mcls.*, rmap.slug "
        "FROM mapclasses AS mcls "
        "INNER JOIN refmaps AS rmap "
        "ON mcls.mapfk = rmap.mapid "
        f"WHERE rmap.slug = '{lulcmap}'"
    ") AS tcls "
    "ON pxcls.clsid = tcls.classid"
)

for i in range(levels):
    if i == 0 or i == 1:
        rootid = "px.cid AS rootid" if not i else "px.rootid"
        lvl = "1" if not i else "2"

        q = (
            f"SELECT {rootid}, ST_Area(geom) AS geom, pxcls.classid, "
            "pxcls.slug, pxcls.code, pxcls.classname "
            "FROM gthruth_fishnet AS px "
            f"INNER JOIN ({cls_view}) AS pxcls "
            "ON px.cid = pxcls.pxid "
            f"WHERE px.level = {lvl}"
        )
    else:
        q = (
            "SELECT px.rootid, px.geom, pxcls.classid, "
            "pxcls.slug, pxcls.code, pxcls.classname "
            f"FROM ({lquery[i-1]}) AS px "
            f"INNER JOIN ({cls_view}) AS pxcls "
            f"ON px.root{str(i+1)} = pxcls.pxid"
        )

    flq.append(q)

_fq = " UNION ALL ".join(flq)

fq = f"SELECT * FROM ({_fq}) AS allview ORDER BY allview.rootid"

In [8]:
hq = (
    "SELECT rootid, refid, refname, "
    "SUM(geom) AS geom "
    "FROM ("
        "SELECT rootid, geom, classid, slug "
        f"code, classname, {refid}, {refname} "
        f"FROM ({fq}) AS foo "
    ") AS foo2 "
    "GROUP BY rootid, refid, refname "
    "ORDER BY rootid, refid"
)

sq = (
    "SELECT rootid, ARRAY_AGG(refid) AS refid, "
    "ARRAY_AGG(refname) AS refname, "
    "ARRAY_AGG(geom) AS geom "
    f"FROM ({hq}) AS foo3 "
    "GROUP BY rootid"
)

In [9]:
px = pp_df = q_to_obj(db, sq)

In [10]:
obj_to_tbl(px, out_tbl)

'/mnt/d/autocls_e4/ground_truth_exp4.xlsx'

In [11]:
px

,rootid,refid,refname,geom
0,1,[3],[Tree cover],[100.0]
1,2,[3],[Tree cover],[100.0]
2,3,[3],[Tree cover],[100.0]
3,4,"[3, 4]","[Tree cover, Shrubland]","[37.5, 62.5]"
4,5,[4],[Shrubland],[100.0]
...,...,...,...,...
61657,61658,[4],[Shrubland],[100.0]
61658,61659,[4],[Shrubland],[100.0]
61659,61660,[4],[Shrubland],[100.0]
61660,61661,[4],[Shrubland],[100.0]
